In [1]:
import cv2
import numpy as np
import math

In [2]:
vid=cv2.VideoCapture(0)

In [3]:
while (1):
    try:
        ret,frame=vid.read()
        frame=cv2.flip(frame,1)
        kernel=np.ones((3,3),np.uint8)

        # şimdi kameranın elimi okumasını istediği yeri oluşturacağız
        roi=frame[100:350,100:350]
        cv2.rectangle(frame,(100,100),(350,350),(255,0,0),1)

        # mask işleminde hsv daha kullanışlı olduğundan roi bölgesindeki renkleri hse formatına çevireceğiz
        hsv=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
        # şimdi el rengimizi ayırt etmek için alt ve üst değer yazacağız
        lower_skin=np.array([0,20,70],np.uint8)
        upper_skin=np.array([20,255,255],np.uint8)

        # mask uyguluyoruz
        mask=cv2.inRange(hsv,lower_skin,upper_skin)
        # oluşacak karanlık noktaları beyazlarla dolduracağız
        # 1 lerden oluşan 3e 3lük kernel siyah noktaları beyzalaştırır
        mask=cv2.dilate(mask,kernel,iterations=4) # iterations=4 demek 4 kere uygula demek beğenmezsem arttırır ya da azaltabilirim

        # tabi ki gürültü de oluşacak ve bunları blurlamam lazım
        mask=cv2.GaussianBlur(mask,(5,5),100)    # kullanılacak blur olarak bunu seçmiş diğerlerini de kullanabilirdik , 100 default bir değer

        # maskı kullanarak sınır çizgileriyle contoursları bulacağız
        contours,_=cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) 
        # şimdi contoursların max alanını belirleyeceğiz
        cnt=max(contours,key=lambda x:cv2.contourArea(x)) # max() inputların en büyüğünü autput olarak verir
        #2 opsiyonel 1 zorunlu değer alır

        # şimdi contours lara biraz daha yaklaşıp sınır çizgilerini çizdireceğiz
        # yazacağım şeyler denenmiş ve kabul edilmiş şeylerdir
        epsilon=0.0005*cv2.arcLength(cnt,True)
        approx=cv2.approxPolyDP(cnt,epsilon,True)

        # şimdi elimizin çevresinde dışbükey örtü oluşturacağız
        hull=cv2.convexHull(cnt)

        # şimdi hull ile oluşam alanı hesaplayacağız yani oluşan şeklin ve elimizin alanını hesaplayacağız
        areaHull=cv2.contourArea(hull)
        areaCnt=cv2.contourArea(cnt) # yani elim

        # şimdi örtü yani hull içerisinde elimizin olmadığı alanın yüzde kaç olduğunu bulacağız
        areaRatio=((areaHull-areaCnt)/areaCnt)*100 # matematiği

        # dışbükey sonucu oluşan kusurları bulma he hesaplama
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull) # bunların ayrıntılı hesaplaması googledeki klasöründe var
        # şimdi convek kusur ile kusurları saydırmadan önce kusur sayımıza 0 diyelim ileride kulanacağız
        l=0

        # şimdi defect içerisindeki kusur değerlerini tek tek atayacağız çünkü oralara çizim yapacağız
        for i in range(defects.shape[0]):
            s,e,f,d=defects[i,0] # tek tek defects değerlerini atamış oldum
            # şimdi bunları tuple içinde approxtaki istediğimiz değerini atayacağız
            start=tuple(approx[s][0])
            end=tuple(approx[e][0])
            far=tuple(approx[f][0])
            # lini 2 yaptığını düşün başlangıç ve biriş noktalarını bildiğimden üçgenin tüm kenar uzunluklarına ulaşabilirim
            # ilk noktaların farkından 1. kenar uzunluğu son noktaların farkından 2. kenar uzunluğu bunların karelerinin toplamının karekökü ile 3. kenarın uzunluğu
            a=math.sqrt((end[0]-start[0])**2+(end[1]-start[1])**2)
            b=math.sqrt((far[0]-start[0])**2+(far[1]-start[1])**2)
            c=math.sqrt((end[0]-far[0])**2+(end[1]-far[1])**2)

            # şimdi de oluşan üçgenin alanını bulacağız
            # a, b ve c üçgenin kenar uzunlukları ve s üçgenin çevre uzunluğunun yarısı olmak üzere;
            # Alan = √(s(s-a)(s-b)(s-c)) formulünden
            s=(a+b+c)/2
            area=(s*(s-a)*(s-b)*(s-c))
            # noktalar ve dışbükey arasındaki mesafe için
            d=(2*area)/a # ?

            # cos teoremiyle 2 doğru arasındaki açıyı bulacağız
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            # şimdi eğer açı 90dan küçük ve 30dan büyükse kusur var deyip kusur sayımı 1 arttıracağım
            if angle <= 90 and d>30:
                l += 1
                # roimdeki far bölgelerime circle koyacağım
                cv2.circle(roi, far, 3, [255,0,0], -1) # (255,0,0) yerine köşeli parantezle yaptı iki türlü de aynı çalıştı
            # başlangıç ve bitiş noktalarını kullanarak çizgi çizeceğiz
            cv2.line(roi,start, end, [0,255,0], 2)
        l+=1
        font = cv2.FONT_HERSHEY_SIMPLEX
        # şimdi areaCnt ile karşılaştırma için kullanacağım değerler denenip bulunmuş değerler
        # kusur sayısına göre kaç parmak gözüktüğünü anladığımızdan artık if elif else
        # bulduğum kusurlara yüklediğim anlamlara göre yazdırıyorum
        if l==1:
            if areaCnt<2000: # elimin orada olmaması durumu # areaCnt=elimdi
                cv2.putText(frame,'Put your hand in the box',(0,50), font, 1, (0,0,255), 3, cv2.LINE_AA)
            else:
                if areaRatio<12: # areaRatio elimin bulunmadığı alanın yüzdesiydi # areaRatio<12 demek orada 1 el var demek dolmuş yani # 12 denenmiş değer
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif areaRatio<17.5:
                    cv2.putText(frame,'Best luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

        elif l==3:
              if areaRatio<27: # bu denneip bulunmuş bir şey
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
              else:# 3 parmak harici bir şey olduğundan ok diye yazalım dedik
                    cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)  
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        elif l==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

        else :
            cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
    except:
        print("You gat a problem")
        
    # durdurmak için başka bir yazım şekli kullandı bunu da bilelim bu da bulunsun
    k = cv2.waitKey(5) & 0xFF # esc basınca çıkacak
    if k == 27:
        break

vid.release() 
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ibrahim/.local/lib/python3.10/site-packages/cv2/qt/plugins"
